In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import re
import sklearn.model_selection as model_selection
import zipfile
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")


# -------------------- Load Dataset ------------------------

dataset_dir = "./dataset3"

imgs = [] #list image matrix
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))

            # Augmentation
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resize = cv2.resize(gray, (150,150))
            # flip_horizontal = cv2.flip(resize, 1)
            # flip_vertical = cv2.flip(resize, 0)
            # flip_both = cv2.flip(resize, -1)

            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(sub_folder)[0]))
            descs.append(normalize_desc(folder, sub_folder))

            print_progress(i, len_sub_folder, folder, sub_folder, filename)


[##########] folder : Animal/Elephant/ ----> file : Elephant_999.jpgg


In [3]:
from skimage.feature import graycomatrix, graycoprops

# ----------------- calculate graycomatrix() & graycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[1], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):

    glcm = graycomatrix(img,
                        distances=dists,
                        angles=agls,
                        levels=lvl,
                        symmetric=sym,
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in graycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label)

    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels):
    glcm_all_agls.append(
            calc_glcm_all_agls(img,
                                label,
                                props=properties)
                            )

columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)

columns.append("label")

In [4]:
# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls,
                      columns = columns)

glcm_df.head()

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,14.774362,14.820053,11.988054,16.363587,0.920517,0.919277,0.946848,0.910407,0.116719,0.110518,...,531.134453,0.000303,0.000286,0.000388,0.000263,0.017394,0.016922,0.019707,0.016203,Elephant
1,18.230112,22.338949,19.504251,22.459619,0.885476,0.830818,0.863465,0.829850,0.108732,0.089391,...,1272.198369,0.000162,0.000138,0.000171,0.000139,0.012733,0.011765,0.013060,0.011784,Elephant
2,27.033110,28.353317,21.542953,28.026440,0.722814,0.694220,0.821865,0.703742,0.045739,0.044401,...,1413.731589,0.000075,0.000073,0.000087,0.000074,0.008681,0.008571,0.009351,0.008611,Elephant
3,8.532125,13.224314,10.315481,12.725508,0.950356,0.910238,0.946020,0.916983,0.389362,0.266580,...,639.189000,0.003012,0.001867,0.002033,0.001955,0.054884,0.043204,0.045093,0.044221,Elephant
4,14.953691,15.997117,11.105548,16.219179,0.920451,0.909988,0.952077,0.905118,0.261319,0.246520,...,838.068916,0.002514,0.002022,0.002293,0.001982,0.050136,0.044965,0.047885,0.044519,Elephant


In [5]:
glcm_df.to_csv("glcm_animal_dataset.csv")

In [6]:
glcm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dissimilarity_0    2000 non-null   float64
 1   dissimilarity_45   2000 non-null   float64
 2   dissimilarity_90   2000 non-null   float64
 3   dissimilarity_135  2000 non-null   float64
 4   correlation_0      2000 non-null   float64
 5   correlation_45     2000 non-null   float64
 6   correlation_90     2000 non-null   float64
 7   correlation_135    2000 non-null   float64
 8   homogeneity_0      2000 non-null   float64
 9   homogeneity_45     2000 non-null   float64
 10  homogeneity_90     2000 non-null   float64
 11  homogeneity_135    2000 non-null   float64
 12  contrast_0         2000 non-null   float64
 13  contrast_45        2000 non-null   float64
 14  contrast_90        2000 non-null   float64
 15  contrast_135       2000 non-null   float64
 16  ASM_0              2000 

In [7]:
glcm0 = glcm_df.drop(["dissimilarity_45", "dissimilarity_90", "dissimilarity_135", "correlation_45", "correlation_90", "correlation_135", "homogeneity_45", "homogeneity_90", "homogeneity_135", "contrast_45", "contrast_90", "contrast_135", "ASM_45", "ASM_90", "ASM_135", "energy_45", "energy_90", "energy_135"], axis="columns")
glcm0.head()

,dissimilarity_0,correlation_0,homogeneity_0,contrast_0,ASM_0,energy_0,label
0,14.774362,0.920517,0.116719,474.791812,0.000303,0.017394,Elephant
1,18.230112,0.885476,0.108732,857.073065,0.000162,0.012733,Elephant
2,27.033110,0.722814,0.045739,1323.750246,0.000075,0.008681,Elephant
3,8.532125,0.950356,0.389362,389.672528,0.003012,0.054884,Elephant
4,14.953691,0.920451,0.261319,703.156644,0.002514,0.050136,Elephant


In [8]:
glcm45 = glcm_df.drop(["dissimilarity_0", "dissimilarity_90", "dissimilarity_135", "correlation_0", "correlation_90", "correlation_135", "homogeneity_0", "homogeneity_90", "homogeneity_135", "contrast_0", "contrast_90", "contrast_135", "ASM_0", "ASM_90", "ASM_135", "energy_0", "energy_90", "energy_135"], axis="columns")
glcm45.head()

,dissimilarity_45,correlation_45,homogeneity_45,contrast_45,ASM_45,energy_45,label
0,14.820053,0.919277,0.110518,478.557182,0.000286,0.016922,Elephant
1,22.338949,0.830818,0.089391,1264.962434,0.000138,0.011765,Elephant
2,28.353317,0.694220,0.044401,1459.127967,0.000073,0.008571,Elephant
3,13.224314,0.910238,0.266580,691.127472,0.001867,0.043204,Elephant
4,15.997117,0.909988,0.246520,795.072204,0.002022,0.044965,Elephant


In [9]:
glcm90 = glcm_df.drop(["dissimilarity_45", "dissimilarity_0", "dissimilarity_135", "correlation_45", "correlation_0", "correlation_135", "homogeneity_45", "homogeneity_0", "homogeneity_135", "contrast_45", "contrast_0", "contrast_135", "ASM_45", "ASM_0", "ASM_135", "energy_45", "energy_0", "energy_135"], axis="columns")
glcm90.head()

,dissimilarity_90,correlation_90,homogeneity_90,contrast_90,ASM_90,energy_90,label
0,11.988054,0.946848,0.145913,314.667785,0.000388,0.019707,Elephant
1,19.504251,0.863465,0.109826,1021.737808,0.000171,0.013060,Elephant
2,21.542953,0.821865,0.054537,850.973647,0.000087,0.009351,Elephant
3,10.315481,0.946020,0.300357,414.952260,0.002033,0.045093,Elephant
4,11.105548,0.952077,0.277577,422.793333,0.002293,0.047885,Elephant


In [10]:
glcm135 = glcm_df.drop(["dissimilarity_45", "dissimilarity_90", "dissimilarity_0", "correlation_45", "correlation_90", "correlation_0", "homogeneity_45", "homogeneity_90", "homogeneity_0", "contrast_45", "contrast_90", "contrast_0", "ASM_45", "ASM_90", "ASM_0", "energy_45", "energy_90", "energy_0"], axis="columns")
glcm135.head()

,dissimilarity_135,correlation_135,homogeneity_135,contrast_135,ASM_135,energy_135,label
0,16.363587,0.910407,0.101641,531.134453,0.000263,0.016203,Elephant
1,22.459619,0.829850,0.091136,1272.198369,0.000139,0.011784,Elephant
2,28.026440,0.703742,0.043366,1413.731589,0.000074,0.008611,Elephant
3,12.725508,0.916983,0.285699,639.189000,0.001955,0.044221,Elephant
4,16.219179,0.905118,0.240104,838.068916,0.001982,0.044519,Elephant


In [11]:
glcm0_45 = glcm_df.drop(["dissimilarity_90", "dissimilarity_135", "correlation_90", "correlation_135", "homogeneity_90", "homogeneity_135", "contrast_90", "contrast_135", "ASM_90", "ASM_135", "energy_90", "energy_135"], axis="columns")
glcm0_45.head()

,dissimilarity_0,dissimilarity_45,correlation_0,correlation_45,homogeneity_0,homogeneity_45,contrast_0,contrast_45,ASM_0,ASM_45,energy_0,energy_45,label
0,14.774362,14.820053,0.920517,0.919277,0.116719,0.110518,474.791812,478.557182,0.000303,0.000286,0.017394,0.016922,Elephant
1,18.230112,22.338949,0.885476,0.830818,0.108732,0.089391,857.073065,1264.962434,0.000162,0.000138,0.012733,0.011765,Elephant
2,27.033110,28.353317,0.722814,0.694220,0.045739,0.044401,1323.750246,1459.127967,0.000075,0.000073,0.008681,0.008571,Elephant
3,8.532125,13.224314,0.950356,0.910238,0.389362,0.266580,389.672528,691.127472,0.003012,0.001867,0.054884,0.043204,Elephant
4,14.953691,15.997117,0.920451,0.909988,0.261319,0.246520,703.156644,795.072204,0.002514,0.002022,0.050136,0.044965,Elephant


In [12]:
glcm0_90 = glcm_df.drop(["dissimilarity_45", "dissimilarity_135", "correlation_45", "correlation_135", "homogeneity_45", "homogeneity_135", "contrast_45", "contrast_135", "ASM_45", "ASM_135", "energy_45", "energy_135"], axis="columns")
glcm0_90.head()

,dissimilarity_0,dissimilarity_90,correlation_0,correlation_90,homogeneity_0,homogeneity_90,contrast_0,contrast_90,ASM_0,ASM_90,energy_0,energy_90,label
0,14.774362,11.988054,0.920517,0.946848,0.116719,0.145913,474.791812,314.667785,0.000303,0.000388,0.017394,0.019707,Elephant
1,18.230112,19.504251,0.885476,0.863465,0.108732,0.109826,857.073065,1021.737808,0.000162,0.000171,0.012733,0.013060,Elephant
2,27.033110,21.542953,0.722814,0.821865,0.045739,0.054537,1323.750246,850.973647,0.000075,0.000087,0.008681,0.009351,Elephant
3,8.532125,10.315481,0.950356,0.946020,0.389362,0.300357,389.672528,414.952260,0.003012,0.002033,0.054884,0.045093,Elephant
4,14.953691,11.105548,0.920451,0.952077,0.261319,0.277577,703.156644,422.793333,0.002514,0.002293,0.050136,0.047885,Elephant


In [13]:
glcm0_135 = glcm_df.drop(["dissimilarity_90", "dissimilarity_45", "correlation_90", "correlation_45", "homogeneity_90", "homogeneity_45", "contrast_90", "contrast_45", "ASM_90", "ASM_45", "energy_90", "energy_45"], axis="columns")
glcm0_135.head()

,dissimilarity_0,dissimilarity_135,correlation_0,correlation_135,homogeneity_0,homogeneity_135,contrast_0,contrast_135,ASM_0,ASM_135,energy_0,energy_135,label
0,14.774362,16.363587,0.920517,0.910407,0.116719,0.101641,474.791812,531.134453,0.000303,0.000263,0.017394,0.016203,Elephant
1,18.230112,22.459619,0.885476,0.829850,0.108732,0.091136,857.073065,1272.198369,0.000162,0.000139,0.012733,0.011784,Elephant
2,27.033110,28.026440,0.722814,0.703742,0.045739,0.043366,1323.750246,1413.731589,0.000075,0.000074,0.008681,0.008611,Elephant
3,8.532125,12.725508,0.950356,0.916983,0.389362,0.285699,389.672528,639.189000,0.003012,0.001955,0.054884,0.044221,Elephant
4,14.953691,16.219179,0.920451,0.905118,0.261319,0.240104,703.156644,838.068916,0.002514,0.001982,0.050136,0.044519,Elephant


In [14]:
glcm45_90 = glcm_df.drop(["dissimilarity_0", "dissimilarity_135", "correlation_0", "correlation_135", "homogeneity_0", "homogeneity_135", "contrast_0", "contrast_135", "ASM_0", "ASM_135", "energy_0", "energy_135"], axis="columns")
glcm45_90.head()

,dissimilarity_45,dissimilarity_90,correlation_45,correlation_90,homogeneity_45,homogeneity_90,contrast_45,contrast_90,ASM_45,ASM_90,energy_45,energy_90,label
0,14.820053,11.988054,0.919277,0.946848,0.110518,0.145913,478.557182,314.667785,0.000286,0.000388,0.016922,0.019707,Elephant
1,22.338949,19.504251,0.830818,0.863465,0.089391,0.109826,1264.962434,1021.737808,0.000138,0.000171,0.011765,0.013060,Elephant
2,28.353317,21.542953,0.694220,0.821865,0.044401,0.054537,1459.127967,850.973647,0.000073,0.000087,0.008571,0.009351,Elephant
3,13.224314,10.315481,0.910238,0.946020,0.266580,0.300357,691.127472,414.952260,0.001867,0.002033,0.043204,0.045093,Elephant
4,15.997117,11.105548,0.909988,0.952077,0.246520,0.277577,795.072204,422.793333,0.002022,0.002293,0.044965,0.047885,Elephant


In [15]:
glcm45_135 = glcm_df.drop(["dissimilarity_0", "dissimilarity_90", "correlation_0", "correlation_90", "homogeneity_0", "homogeneity_90", "contrast_0", "contrast_90", "ASM_0", "ASM_90", "energy_0", "energy_90"], axis="columns")
glcm45_135.head()

,dissimilarity_45,dissimilarity_135,correlation_45,correlation_135,homogeneity_45,homogeneity_135,contrast_45,contrast_135,ASM_45,ASM_135,energy_45,energy_135,label
0,14.820053,16.363587,0.919277,0.910407,0.110518,0.101641,478.557182,531.134453,0.000286,0.000263,0.016922,0.016203,Elephant
1,22.338949,22.459619,0.830818,0.829850,0.089391,0.091136,1264.962434,1272.198369,0.000138,0.000139,0.011765,0.011784,Elephant
2,28.353317,28.026440,0.694220,0.703742,0.044401,0.043366,1459.127967,1413.731589,0.000073,0.000074,0.008571,0.008611,Elephant
3,13.224314,12.725508,0.910238,0.916983,0.266580,0.285699,691.127472,639.189000,0.001867,0.001955,0.043204,0.044221,Elephant
4,15.997117,16.219179,0.909988,0.905118,0.246520,0.240104,795.072204,838.068916,0.002022,0.001982,0.044965,0.044519,Elephant


In [16]:
glcm90_135 = glcm_df.drop(["dissimilarity_0", "dissimilarity_45", "correlation_0", "correlation_45", "homogeneity_0", "homogeneity_45", "contrast_0", "contrast_45", "ASM_0", "ASM_45", "energy_0", "energy_45"], axis="columns")
glcm90_135.head()

,dissimilarity_90,dissimilarity_135,correlation_90,correlation_135,homogeneity_90,homogeneity_135,contrast_90,contrast_135,ASM_90,ASM_135,energy_90,energy_135,label
0,11.988054,16.363587,0.946848,0.910407,0.145913,0.101641,314.667785,531.134453,0.000388,0.000263,0.019707,0.016203,Elephant
1,19.504251,22.459619,0.863465,0.829850,0.109826,0.091136,1021.737808,1272.198369,0.000171,0.000139,0.013060,0.011784,Elephant
2,21.542953,28.026440,0.821865,0.703742,0.054537,0.043366,850.973647,1413.731589,0.000087,0.000074,0.009351,0.008611,Elephant
3,10.315481,12.725508,0.946020,0.916983,0.300357,0.285699,414.952260,639.189000,0.002033,0.001955,0.045093,0.044221,Elephant
4,11.105548,16.219179,0.952077,0.905118,0.277577,0.240104,422.793333,838.068916,0.002293,0.001982,0.047885,0.044519,Elephant


In [17]:
glcm0_45_90 = glcm_df.drop(["dissimilarity_135","correlation_135", "homogeneity_135", "contrast_135", "ASM_135", "energy_135"], axis="columns")
glcm0_45_90.head()

,dissimilarity_0,dissimilarity_45,dissimilarity_90,correlation_0,correlation_45,correlation_90,homogeneity_0,homogeneity_45,homogeneity_90,contrast_0,contrast_45,contrast_90,ASM_0,ASM_45,ASM_90,energy_0,energy_45,energy_90,label
0,14.774362,14.820053,11.988054,0.920517,0.919277,0.946848,0.116719,0.110518,0.145913,474.791812,478.557182,314.667785,0.000303,0.000286,0.000388,0.017394,0.016922,0.019707,Elephant
1,18.230112,22.338949,19.504251,0.885476,0.830818,0.863465,0.108732,0.089391,0.109826,857.073065,1264.962434,1021.737808,0.000162,0.000138,0.000171,0.012733,0.011765,0.013060,Elephant
2,27.033110,28.353317,21.542953,0.722814,0.694220,0.821865,0.045739,0.044401,0.054537,1323.750246,1459.127967,850.973647,0.000075,0.000073,0.000087,0.008681,0.008571,0.009351,Elephant
3,8.532125,13.224314,10.315481,0.950356,0.910238,0.946020,0.389362,0.266580,0.300357,389.672528,691.127472,414.952260,0.003012,0.001867,0.002033,0.054884,0.043204,0.045093,Elephant
4,14.953691,15.997117,11.105548,0.920451,0.909988,0.952077,0.261319,0.246520,0.277577,703.156644,795.072204,422.793333,0.002514,0.002022,0.002293,0.050136,0.044965,0.047885,Elephant


In [18]:
glcm0_45_135 = glcm_df.drop(["dissimilarity_90", "correlation_90", "homogeneity_90", "contrast_90", "ASM_90", "energy_90"], axis="columns")
glcm0_45_135.head()

,dissimilarity_0,dissimilarity_45,dissimilarity_135,correlation_0,correlation_45,correlation_135,homogeneity_0,homogeneity_45,homogeneity_135,contrast_0,contrast_45,contrast_135,ASM_0,ASM_45,ASM_135,energy_0,energy_45,energy_135,label
0,14.774362,14.820053,16.363587,0.920517,0.919277,0.910407,0.116719,0.110518,0.101641,474.791812,478.557182,531.134453,0.000303,0.000286,0.000263,0.017394,0.016922,0.016203,Elephant
1,18.230112,22.338949,22.459619,0.885476,0.830818,0.829850,0.108732,0.089391,0.091136,857.073065,1264.962434,1272.198369,0.000162,0.000138,0.000139,0.012733,0.011765,0.011784,Elephant
2,27.033110,28.353317,28.026440,0.722814,0.694220,0.703742,0.045739,0.044401,0.043366,1323.750246,1459.127967,1413.731589,0.000075,0.000073,0.000074,0.008681,0.008571,0.008611,Elephant
3,8.532125,13.224314,12.725508,0.950356,0.910238,0.916983,0.389362,0.266580,0.285699,389.672528,691.127472,639.189000,0.003012,0.001867,0.001955,0.054884,0.043204,0.044221,Elephant
4,14.953691,15.997117,16.219179,0.920451,0.909988,0.905118,0.261319,0.246520,0.240104,703.156644,795.072204,838.068916,0.002514,0.002022,0.001982,0.050136,0.044965,0.044519,Elephant


In [19]:
glcm0_90_135 = glcm_df.drop(["dissimilarity_45", "correlation_45", "homogeneity_45", "contrast_45", "ASM_45", "energy_45"], axis="columns")
glcm0_90_135.head()

,dissimilarity_0,dissimilarity_90,dissimilarity_135,correlation_0,correlation_90,correlation_135,homogeneity_0,homogeneity_90,homogeneity_135,contrast_0,contrast_90,contrast_135,ASM_0,ASM_90,ASM_135,energy_0,energy_90,energy_135,label
0,14.774362,11.988054,16.363587,0.920517,0.946848,0.910407,0.116719,0.145913,0.101641,474.791812,314.667785,531.134453,0.000303,0.000388,0.000263,0.017394,0.019707,0.016203,Elephant
1,18.230112,19.504251,22.459619,0.885476,0.863465,0.829850,0.108732,0.109826,0.091136,857.073065,1021.737808,1272.198369,0.000162,0.000171,0.000139,0.012733,0.013060,0.011784,Elephant
2,27.033110,21.542953,28.026440,0.722814,0.821865,0.703742,0.045739,0.054537,0.043366,1323.750246,850.973647,1413.731589,0.000075,0.000087,0.000074,0.008681,0.009351,0.008611,Elephant
3,8.532125,10.315481,12.725508,0.950356,0.946020,0.916983,0.389362,0.300357,0.285699,389.672528,414.952260,639.189000,0.003012,0.002033,0.001955,0.054884,0.045093,0.044221,Elephant
4,14.953691,11.105548,16.219179,0.920451,0.952077,0.905118,0.261319,0.277577,0.240104,703.156644,422.793333,838.068916,0.002514,0.002293,0.001982,0.050136,0.047885,0.044519,Elephant


In [20]:
glcm45_90_135 = glcm_df.drop(["dissimilarity_0", "correlation_0", "homogeneity_0", "contrast_0", "ASM_0", "energy_0"], axis="columns")
glcm45_90_135.head()

,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_45,correlation_90,correlation_135,homogeneity_45,homogeneity_90,homogeneity_135,contrast_45,contrast_90,contrast_135,ASM_45,ASM_90,ASM_135,energy_45,energy_90,energy_135,label
0,14.820053,11.988054,16.363587,0.919277,0.946848,0.910407,0.110518,0.145913,0.101641,478.557182,314.667785,531.134453,0.000286,0.000388,0.000263,0.016922,0.019707,0.016203,Elephant
1,22.338949,19.504251,22.459619,0.830818,0.863465,0.829850,0.089391,0.109826,0.091136,1264.962434,1021.737808,1272.198369,0.000138,0.000171,0.000139,0.011765,0.013060,0.011784,Elephant
2,28.353317,21.542953,28.026440,0.694220,0.821865,0.703742,0.044401,0.054537,0.043366,1459.127967,850.973647,1413.731589,0.000073,0.000087,0.000074,0.008571,0.009351,0.008611,Elephant
3,13.224314,10.315481,12.725508,0.910238,0.946020,0.916983,0.266580,0.300357,0.285699,691.127472,414.952260,639.189000,0.001867,0.002033,0.001955,0.043204,0.045093,0.044221,Elephant
4,15.997117,11.105548,16.219179,0.909988,0.952077,0.905118,0.246520,0.277577,0.240104,795.072204,422.793333,838.068916,0.002022,0.002293,0.001982,0.044965,0.047885,0.044519,Elephant


In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix 

**GLCM SUDUT 0**

In [22]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0 = glcm0[glcm0.columns[:6]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0 = glcm0['label']

X0_train, X0_test, y0_train, y0_test = model_selection.train_test_split(X0, y0, train_size=0.70, test_size=0.30, random_state=101)

In [23]:
# train the model on train set 
model = SVC() 
model.fit(X0_train, y0_train) 
  
# print prediction results 
predictions = model.predict(X0_test) 
print(classification_report(y0_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.91      0.84       289
       Zebra       0.90      0.77      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [24]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0.fit(X0_train, y0_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.779 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [25]:
# print best parameter after tuning 
print(grid0.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0.best_estimator_) 

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1000, gamma=0.0001)


In [26]:
grid_predictions = grid0.predict(X0_test) 
  
# print classification report 
print(classification_report(y0_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.82      0.93      0.87       289
       Zebra       0.92      0.81      0.86       311

    accuracy                           0.87       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.87      0.87       600



**GLCM SUDUT 45**

In [27]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X45 = glcm45[glcm45.columns[:6]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y45 = glcm45['label']

X45_train, X45_test, y45_train, y45_test = model_selection.train_test_split(X45, y45, train_size=0.70, test_size=0.30, random_state=101)

In [28]:
# train the model on train set 
model = SVC() 
model.fit(X45_train, y45_train) 
  
# print prediction results 
predictions = model.predict(X45_test) 
print(classification_report(y45_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.90      0.84       289
       Zebra       0.89      0.77      0.82       311

    accuracy                           0.83       600
   macro avg       0.84      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



In [29]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid45 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid45.fit(X45_train, y45_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.757 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [30]:
# print best parameter after tuning 
print(grid45.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid45.best_estimator_) 

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=100, gamma=0.0001)


In [31]:
grid_predictions = grid45.predict(X45_test) 
  
# print classification report 
print(classification_report(y45_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.81      0.89      0.85       289
       Zebra       0.88      0.81      0.85       311

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.85      0.85      0.85       600



**GLCM SUDUT 90**

In [32]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X90 = glcm90[glcm90.columns[:6]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y90 = glcm90['label']

X90_train, X90_test, y90_train, y90_test = model_selection.train_test_split(X90, y90, train_size=0.70, test_size=0.30, random_state=101)

In [33]:
# train the model on train set 
model = SVC() 
model.fit(X90_train, y90_train) 
  
# print prediction results 
predictions = model.predict(X90_test) 
print(classification_report(y90_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.73      0.74      0.74       289
       Zebra       0.76      0.74      0.75       311

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600



In [34]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid90 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid90.fit(X90_train, y90_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.514 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.496 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.500 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.675 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [ ]:
# print best parameter after tuning 
print(grid90.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid90.best_estimator_) 

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1000, gamma=0.0001)


In [ ]:
grid_predictions = grid90.predict(X90_test) 
  
# print classification report 
print(classification_report(y90_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.75      0.78      0.77       289
       Zebra       0.79      0.76      0.77       311

    accuracy                           0.77       600
   macro avg       0.77      0.77      0.77       600
weighted avg       0.77      0.77      0.77       600



**GLCM SUDUT 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X135 = glcm135[glcm135.columns[:6]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y135 = glcm135['label']

X135_train, X135_test, y135_train, y135_test = model_selection.train_test_split(X135, y135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
# train the model on train set 
model = SVC() 
model.fit(X135_train, y135_train) 
  
# print prediction results 
predictions = model.predict(X135_test) 
print(classification_report(y135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.77      0.90      0.83       289
       Zebra       0.89      0.75      0.81       311

    accuracy                           0.82       600
   macro avg       0.83      0.82      0.82       600
weighted avg       0.83      0.82      0.82       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid135.fit(X135_train, y135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.754 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid135.best_estimator_) 

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1000, gamma=0.0001)


In [ ]:
grid_predictions = grid135.predict(X135_test) 
  
# print classification report 
print(classification_report(y135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.88      0.83       289
       Zebra       0.88      0.78      0.82       311

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600



**GLCM SUDUT 0 DAN 45**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_45 = glcm0_45[glcm0_45.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_45 = glcm0_45['label']

X0_45_train, X0_45_test, y0_45_train, y0_45_test = model_selection.train_test_split(X0_45, y0_45, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_45_train, y0_45_train) 
  
# print prediction results 
predictions = model.predict(X0_45_test) 
print(classification_report(y0_45_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.90      0.84       289
       Zebra       0.89      0.77      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_45 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_45.fit(X0_45_train, y0_45_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_45.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_45.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_45.predict(X0_45_test) 
  
# print classification report 
print(classification_report(y0_45_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.80      0.89      0.84       289
       Zebra       0.88      0.79      0.84       311

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600



**GLCM SUDUT 0 DAN 90**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_90 = glcm0_90[glcm0_90.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_90 = glcm0_90['label']

X0_90_train, X0_90_test, y0_90_train, y0_90_test = model_selection.train_test_split(X0_90, y0_90, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_90_train, y0_90_train) 
  
# print prediction results 
predictions = model.predict(X0_90_test) 
print(classification_report(y0_90_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.90      0.84       289
       Zebra       0.90      0.77      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_90 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_90.fit(X0_90_train, y0_90_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_90.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_90.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_90.predict(X0_90_test) 
  
# print classification report 
print(classification_report(y0_90_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.89      0.84       289
       Zebra       0.89      0.78      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



**GLCM SUDUT 0 DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_135 = glcm0_135[glcm0_135.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_135 = glcm0_135['label']

X0_135_train, X0_135_test, y0_135_train, y0_135_test = model_selection.train_test_split(X0_135, y0_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_135_train, y0_135_train) 
  
# print prediction results 
predictions = model.predict(X0_135_test) 
print(classification_report(y0_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.90      0.84       289
       Zebra       0.89      0.77      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_135.fit(X0_135_train, y0_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_135.predict(X0_135_test) 
  
# print classification report 
print(classification_report(y0_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.88      0.83       289
       Zebra       0.88      0.78      0.83       311

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



**GLCM SUDUT 45 DAN 90**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X45_90 = glcm45_90[glcm45_90.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y45_90 = glcm45_90['label']

X45_90_train, X45_90_test, y45_90_train, y45_90_test = model_selection.train_test_split(X45_90, y45_90, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X45_90_train, y45_90_train) 
  
# print prediction results 
predictions = model.predict(X45_90_test) 
print(classification_report(y45_90_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.91      0.84       289
       Zebra       0.90      0.76      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid45_90 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid45_90.fit(X45_90_train, y45_90_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid45_90.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid45_90.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid45_90.predict(X45_90_test) 
  
# print classification report 
print(classification_report(y45_90_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.80      0.90      0.84       289
       Zebra       0.89      0.78      0.84       311

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.85      0.84      0.84       600



**GLCM SUDUT 45 DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X45_135 = glcm45_135[glcm45_135.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y45_135 = glcm45_135['label']

X45_135_train, X45_135_test, y45_135_train, y45_135_test = model_selection.train_test_split(X45_135, y45_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X45_135_train, y45_135_train) 
  
# print prediction results 
predictions = model.predict(X45_135_test) 
print(classification_report(y45_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.76      0.90      0.83       289
       Zebra       0.89      0.74      0.81       311

    accuracy                           0.82       600
   macro avg       0.83      0.82      0.82       600
weighted avg       0.83      0.82      0.82       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid45_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid45_135.fit(X45_135_train, y45_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid45_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid45_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid45_135.predict(X45_135_test) 
  
# print classification report 
print(classification_report(y45_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.84      0.89      0.86       289
       Zebra       0.89      0.84      0.87       311

    accuracy                           0.86       600
   macro avg       0.87      0.87      0.86       600
weighted avg       0.87      0.86      0.87       600



**GLCM SUDUT 90 DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X90_135 = glcm90_135[glcm90_135.columns[:12]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y90_135 = glcm90_135['label']

X90_135_train, X90_135_test, y90_135_train, y90_135_test = model_selection.train_test_split(X90_135, y90_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X90_135_train, y90_135_train) 
  
# print prediction results 
predictions = model.predict(X90_135_test) 
print(classification_report(y90_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.91      0.84       289
       Zebra       0.90      0.76      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid90_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid90_135.fit(X90_135_train, y90_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid90_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid90_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid90_135.predict(X90_135_test) 
  
# print classification report 
print(classification_report(y90_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.89      0.84       289
       Zebra       0.89      0.78      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



**GLCM SUDUT 0, 45, DAN 90**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_45_90 = glcm0_45_90[glcm0_45_90.columns[:18]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_45_90 = glcm0_45_90['label']

X0_45_90_train, X0_45_90_test, y0_45_90_train, y0_45_90_test = model_selection.train_test_split(X0_45_90, y0_45_90, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_45_90_train, y0_45_90_train) 
  
# print prediction results 
predictions = model.predict(X0_45_90_test) 
print(classification_report(y0_45_90_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.77      0.91      0.83       289
       Zebra       0.90      0.75      0.82       311

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_45_90 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_45_90.fit(X0_45_90_train, y0_45_90_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_45_90.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_45_90.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_45_90.predict(X0_45_90_test) 
  
# print classification report 
print(classification_report(y0_45_90_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.80      0.90      0.85       289
       Zebra       0.89      0.79      0.84       311

    accuracy                           0.84       600
   macro avg       0.85      0.85      0.84       600
weighted avg       0.85      0.84      0.84       600



**GLCM SUDUT 0, 45, DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_45_135 = glcm0_45_135[glcm0_45_135.columns[:18]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_45_135 = glcm0_45_135['label']

X0_45_135_train, X0_45_135_test, y0_45_135_train, y0_45_135_test = model_selection.train_test_split(X0_45_135, y0_45_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_45_135_train, y0_45_135_train) 
  
# print prediction results 
predictions = model.predict(X0_45_135_test) 
print(classification_report(y0_45_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.90      0.84       289
       Zebra       0.90      0.77      0.83       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_45_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_45_135.fit(X0_45_135_train, y0_45_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_45_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_45_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_45_135.predict(X0_45_135_test) 
  
# print classification report 
print(classification_report(y0_45_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.82      0.88      0.85       289
       Zebra       0.88      0.82      0.85       311

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.85      0.85      0.85       600



**GLCM SUDUT 0, 90, DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X0_90_135 = glcm0_90_135[glcm0_90_135.columns[:18]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y0_90_135 = glcm0_90_135['label']

X0_90_135_train, X0_90_135_test, y0_90_135_train, y0_90_135_test = model_selection.train_test_split(X0_90_135, y0_90_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X0_90_135_train, y0_90_135_train) 
  
# print prediction results 
predictions = model.predict(X0_90_135_test) 
print(classification_report(y0_90_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.79      0.90      0.84       289
       Zebra       0.90      0.77      0.83       311

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid0_90_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid0_90_135.fit(X0_90_135_train, y0_90_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid0_90_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid0_90_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid0_90_135.predict(X0_90_135_test) 
  
# print classification report 
print(classification_report(y0_90_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.80      0.89      0.84       289
       Zebra       0.89      0.80      0.84       311

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.85      0.84      0.84       600



**GLCM SUDUT 45, 90, DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X45_90_135 = glcm45_90_135[glcm45_90_135.columns[:18]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y45_90_135 = glcm45_90_135['label']

X45_90_135_train, X45_90_135_test, y45_90_135_train, y45_90_135_test = model_selection.train_test_split(X45_90_135, y45_90_135, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X45_90_135_train, y45_90_135_train) 
  
# print prediction results 
predictions = model.predict(X45_90_135_test) 
print(classification_report(y45_90_135_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.92      0.84       289
       Zebra       0.91      0.76      0.82       311

    accuracy                           0.83       600
   macro avg       0.84      0.84      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid45_90_135 = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid45_90_135.fit(X45_90_135_train, y45_90_135_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid45_90_135.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid45_90_135.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid45_90_135.predict(X45_90_135_test) 
  
# print classification report 
print(classification_report(y45_90_135_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.83      0.90      0.86       289
       Zebra       0.90      0.83      0.86       311

    accuracy                           0.86       600
   macro avg       0.86      0.86      0.86       600
weighted avg       0.87      0.86      0.86       600



**GLCM SUDUT 0, 45, 90, DAN 135**

In [ ]:
# memisahkan atribut pada dataset dan menyimpannya pada sebuah variabel
X = glcm_df[glcm_df.columns[:24]]

# memisahkan label pada dataset dan menyimpannya pada sebuah variabel
y = glcm_df['label']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.70, test_size=0.30, random_state=101)

In [ ]:
model = SVC() 
model.fit(X_train, y_train) 
  
# print prediction results 
predictions = model.predict(X_test) 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

    Elephant       0.78      0.91      0.84       289
       Zebra       0.90      0.76      0.82       311

    accuracy                           0.83       600
   macro avg       0.84      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.511 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.0001)


In [ ]:
grid_predictions = grid.predict(X_test) 
  
# print classification report 
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

    Elephant       0.85      0.89      0.87       289
       Zebra       0.90      0.85      0.87       311

    accuracy                           0.87       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.87      0.87       600



In [ ]:
from sklearn.model_selection import KFold, cross_val_score
# Perform k-fold cross-validation
k = KFold(n_splits = 5)  # Number of folds
cv_scores = cross_val_score(grid, X_train, y_train, cv=k)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Predict on test set
predictions = model.predict(X_test)

# Print classification report
print(classification_report(y_test, predictions))